In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from obspy.signal.trigger import classic_sta_lta, recursive_sta_lta
from scipy.signal import stft

#### Load and Preprocess the data
Read the CSV file, apply STA/LTA and STFT transformations, and normalize the data.

In [ ]:
# Load CSV data
data = pd.read_csv('data.csv')

In [ ]:
# Apply STA/LTA
def apply_sta_lta(data, nsta, nlta, method='recursive'):
    if method == 'recursive':
        sta_lta = recursive_sta_lta(data, nsta, nlta)
    elif method == 'classic':
        sta_lta = classic_sta_lta(data, nsta, nlta)
    else:
        raise ValueError("Unsupported STA/LTA method")
    return sta_lta

# Apply STFT
def apply_stft(data, fs=100, nperseg=256):
    f, t, Zxx = stft(data, fs=fs, nperseg=nperseg)
    return np.abs(Zxx)

# Extract features and labels (assuming you have a 'label' column)
X = data['acceleration_x'].values
y = data['label'].values

# Apply STA/LTA
nsta = int(10 * 100)  # Short-term window
nlta = int(60 * 100)  # Long-term window
sta_lta_values = apply_sta_lta(X, nsta, nlta)

# Apply STFT
stft_values = apply_stft(X)

# Combine features
features = np.vstack((sta_lta_values, stft_values.flatten())).T

# Normalize/standardize the data
features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

#### Prepare data for PyTorch
Convert the data into PyTorch tensors and create a DataLoader.


In [ ]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(features, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a dataset and dataloader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#### Define the CNN model
Create a 1D CNN model using PyTorch.


In [ ]:
# Define the CNN model
class SeismicCNN(nn.Module):
    def __init__(self):
        super(SeismicCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * (features.shape[1] // 2 // 2), 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * (features.shape[1] // 2 // 2))
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Instantiate the model, define the loss function and the optimizer
model = SeismicCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Train the model
Train the CNN model on the preprocessed data.

In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.unsqueeze(1)  # Add channel dimension
        labels = labels.unsqueeze(1)  # Add channel dimension

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')


#### Evaluate the model
Test the model on a separate test set

In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.unsqueeze(1)  # Add channel dimension
        labels = labels.unsqueeze(1)  # Add channel dimension
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')